# Candlestick Pattern Strategies

This notebook demonstrates how to create trading strategies based on candlestick patterns:

1. **Bullish/Bearish Engulfing Pattern**
2. **Hammer and Shooting Star**
3. **Doji Pattern**
4. **Morning and Evening Star**
5. **Strategy Comparison**

Candlestick patterns are powerful tools for identifying potential trend reversals or continuations.

In [ ]:
# Install the library (run this cell if using Colab or if you haven't installed the package)
!pip install simple-backtest yfinance

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Any, Dict, List

from simple_backtest import BacktestConfig, Backtest
from simple_backtest.strategy import Strategy
from simple_backtest.visualization import plot_equity_curve

## Load Data

In [2]:
# Download data
ticker = "SPY"  # S&P 500 ETF - good for candlestick patterns
data = yf.download(ticker, start="2020-01-01", end="2023-12-31", progress=False)
data = data.dropna()

print(f"Data shape: {data.shape}")
print(f"Date range: {data.index[0]} to {data.index[-1]}")

/var/folders/9y/pp82h2912h3781myzdncvs_r0000gp/T/ipykernel_55048/3642787627.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2020-01-01", end="2023-12-31", progress=False)


Data shape: (1006, 5)
Date range: 2020-01-02 00:00:00 to 2023-12-29 00:00:00


# Download data
ticker = "SPY"  # S&P 500 ETF - good for candlestick patterns
data = yf.download(ticker, start="2020-01-01", end="2023-12-31", progress=False)

# Handle MultiIndex columns if present
if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.get_level_values(0)

data = data.dropna()

print(f"Data shape: {data.shape}")
print(f"Date range: {data.index[0]} to {data.index[-1]}")

In [3]:
class EngulfingStrategy(Strategy):
    """Trade based on bullish/bearish engulfing patterns."""
    
    def __init__(self, shares: float = 10, name: str = None):
        super().__init__(name=name or "Engulfing")
        self.shares = shares
    
    def is_bullish_engulfing(self, prev_candle, curr_candle):
        """Check if current candle is bullish engulfing."""
        # Previous candle is bearish (red)
        prev_bearish = prev_candle['Close'] < prev_candle['Open']
        # Current candle is bullish (green)
        curr_bullish = curr_candle['Close'] > curr_candle['Open']
        # Current candle engulfs previous
        engulfs = (curr_candle['Open'] < prev_candle['Close'] and 
                   curr_candle['Close'] > prev_candle['Open'])
        
        return prev_bearish and curr_bullish and engulfs
    
    def is_bearish_engulfing(self, prev_candle, curr_candle):
        """Check if current candle is bearish engulfing."""
        # Previous candle is bullish (green)
        prev_bullish = prev_candle['Close'] > prev_candle['Open']
        # Current candle is bearish (red)
        curr_bearish = curr_candle['Close'] < curr_candle['Open']
        # Current candle engulfs previous
        engulfs = (curr_candle['Open'] > prev_candle['Close'] and 
                   curr_candle['Close'] < prev_candle['Open'])
        
        return prev_bullish and curr_bearish and engulfs
    
    def predict(self, data: pd.DataFrame, trade_history: List[Dict[str, Any]]) -> Dict[str, Any]:
        if len(data) < 2:
            return self.hold()
        
        prev_candle = data.iloc[-2]
        curr_candle = data.iloc[-1]
        
        # Bullish engulfing - buy signal
        if self.is_bullish_engulfing(prev_candle, curr_candle) and not self.has_position():
            return self.buy(self.shares)
        
        # Bearish engulfing - sell signal
        if self.is_bearish_engulfing(prev_candle, curr_candle) and self.has_position():
            return self.sell(self.shares)
        
        return self.hold()

print("✓ EngulfingStrategy created")

✓ EngulfingStrategy created


## 2. Hammer and Shooting Star

**Hammer**: Bullish reversal pattern
- Small body at top of candle
- Long lower shadow (2x body size)
- Little to no upper shadow

**Shooting Star**: Bearish reversal pattern
- Small body at bottom of candle
- Long upper shadow (2x body size)
- Little to no lower shadow

In [4]:
class HammerShootingStarStrategy(Strategy):
    """Trade based on hammer and shooting star patterns."""
    
    def __init__(self, shares: float = 10, name: str = None):
        super().__init__(name=name or "Hammer_ShootingStar")
        self.shares = shares
    
    def is_hammer(self, candle):
        """Check if candle is a hammer pattern."""
        body = abs(candle['Close'] - candle['Open'])
        lower_shadow = min(candle['Open'], candle['Close']) - candle['Low']
        upper_shadow = candle['High'] - max(candle['Open'], candle['Close'])
        
        # Long lower shadow (at least 2x body)
        long_lower = lower_shadow >= 2 * body
        # Small upper shadow (less than body)
        small_upper = upper_shadow <= body * 0.3
        # Body in upper part of range
        body_position = (candle['High'] - min(candle['Open'], candle['Close'])) / (candle['High'] - candle['Low'])
        
        return long_lower and small_upper and body_position < 0.4
    
    def is_shooting_star(self, candle):
        """Check if candle is a shooting star pattern."""
        body = abs(candle['Close'] - candle['Open'])
        lower_shadow = min(candle['Open'], candle['Close']) - candle['Low']
        upper_shadow = candle['High'] - max(candle['Open'], candle['Close'])
        
        # Long upper shadow (at least 2x body)
        long_upper = upper_shadow >= 2 * body
        # Small lower shadow (less than body)
        small_lower = lower_shadow <= body * 0.3
        # Body in lower part of range
        body_position = (max(candle['Open'], candle['Close']) - candle['Low']) / (candle['High'] - candle['Low'])
        
        return long_upper and small_lower and body_position < 0.4
    
    def predict(self, data: pd.DataFrame, trade_history: List[Dict[str, Any]]) -> Dict[str, Any]:
        if len(data) < 1:
            return self.hold()
        
        curr_candle = data.iloc[-1]
        
        # Hammer - bullish signal
        if self.is_hammer(curr_candle) and not self.has_position():
            return self.buy(self.shares)
        
        # Shooting star - bearish signal
        if self.is_shooting_star(curr_candle) and self.has_position():
            return self.sell(self.shares)
        
        return self.hold()

print("✓ HammerShootingStarStrategy created")

✓ HammerShootingStarStrategy created


## 3. Doji Pattern

**Doji**: Indecision pattern where open and close prices are nearly equal
- Small body (< 5% of high-low range)
- Signals potential trend reversal
- Trade direction depends on context (trend before doji)

In [5]:
class DojiStrategy(Strategy):
    """Trade based on doji patterns with trend confirmation."""
    
    def __init__(self, shares: float = 10, ma_period: int = 20, name: str = None):
        super().__init__(name=name or "Doji")
        self.shares = shares
        self.ma_period = ma_period
    
    def is_doji(self, candle, threshold: float = 0.05):
        """Check if candle is a doji."""
        body = abs(candle['Close'] - candle['Open'])
        candle_range = candle['High'] - candle['Low']
        
        if candle_range == 0:
            return False
        
        # Body is very small compared to total range
        return (body / candle_range) < threshold
    
    def predict(self, data: pd.DataFrame, trade_history: List[Dict[str, Any]]) -> Dict[str, Any]:
        if len(data) < self.ma_period + 1:
            return self.hold()
        
        curr_candle = data.iloc[-1]
        
        # Calculate trend using moving average
        ma = data['Close'].tail(self.ma_period).mean()
        
        if self.is_doji(curr_candle):
            # Doji in downtrend (price below MA) - potential bullish reversal
            if curr_candle['Close'] < ma and not self.has_position():
                return self.buy(self.shares)
            
            # Doji in uptrend (price above MA) - potential bearish reversal
            if curr_candle['Close'] > ma and self.has_position():
                return self.sell(self.shares)
        
        return self.hold()

print("✓ DojiStrategy created")

✓ DojiStrategy created


## 4. Morning and Evening Star

**Morning Star**: Bullish reversal (3-candle pattern)
1. Long bearish candle
2. Small body candle (gap down)
3. Long bullish candle

**Evening Star**: Bearish reversal (3-candle pattern)
1. Long bullish candle
2. Small body candle (gap up)
3. Long bearish candle

In [6]:
class StarStrategy(Strategy):
    """Trade based on morning and evening star patterns."""
    
    def __init__(self, shares: float = 10, name: str = None):
        super().__init__(name=name or "Star")
        self.shares = shares
    
    def is_morning_star(self, candle1, candle2, candle3):
        """Check for morning star pattern."""
        # First candle: long bearish
        c1_bearish = candle1['Close'] < candle1['Open']
        c1_body = abs(candle1['Close'] - candle1['Open'])
        c1_range = candle1['High'] - candle1['Low']
        c1_long = c1_body > 0.6 * c1_range
        
        # Second candle: small body
        c2_body = abs(candle2['Close'] - candle2['Open'])
        c2_range = candle2['High'] - candle2['Low']
        c2_small = c2_body < 0.3 * c2_range
        
        # Third candle: long bullish
        c3_bullish = candle3['Close'] > candle3['Open']
        c3_body = abs(candle3['Close'] - candle3['Open'])
        c3_range = candle3['High'] - candle3['Low']
        c3_long = c3_body > 0.6 * c3_range
        
        return c1_bearish and c1_long and c2_small and c3_bullish and c3_long
    
    def is_evening_star(self, candle1, candle2, candle3):
        """Check for evening star pattern."""
        # First candle: long bullish
        c1_bullish = candle1['Close'] > candle1['Open']
        c1_body = abs(candle1['Close'] - candle1['Open'])
        c1_range = candle1['High'] - candle1['Low']
        c1_long = c1_body > 0.6 * c1_range
        
        # Second candle: small body
        c2_body = abs(candle2['Close'] - candle2['Open'])
        c2_range = candle2['High'] - candle2['Low']
        c2_small = c2_body < 0.3 * c2_range
        
        # Third candle: long bearish
        c3_bearish = candle3['Close'] < candle3['Open']
        c3_body = abs(candle3['Close'] - candle3['Open'])
        c3_range = candle3['High'] - candle3['Low']
        c3_long = c3_body > 0.6 * c3_range
        
        return c1_bullish and c1_long and c2_small and c3_bearish and c3_long
    
    def predict(self, data: pd.DataFrame, trade_history: List[Dict[str, Any]]) -> Dict[str, Any]:
        if len(data) < 3:
            return self.hold()
        
        candle1 = data.iloc[-3]
        candle2 = data.iloc[-2]
        candle3 = data.iloc[-1]
        
        # Morning star - bullish reversal
        if self.is_morning_star(candle1, candle2, candle3) and not self.has_position():
            return self.buy(self.shares)
        
        # Evening star - bearish reversal
        if self.is_evening_star(candle1, candle2, candle3) and self.has_position():
            return self.sell(self.shares)
        
        return self.hold()

print("✓ StarStrategy created")

✓ StarStrategy created


## 5. Compare All Candlestick Strategies

In [7]:
# Configure backtest
config = BacktestConfig(
    initial_capital=10000.0,
    lookback_period=30,
    commission_type="percentage",
    commission_value=0.001,
    risk_free_rate=0.02
)

# Create all strategies
strategies = [
    EngulfingStrategy(shares=10),
    HammerShootingStarStrategy(shares=10),
    DojiStrategy(shares=10, ma_period=20),
    StarStrategy(shares=10),
]

print("Testing strategies:")
for s in strategies:
    print(f"  - {s.get_name()}")

Testing strategies:
  - Engulfing
  - Hammer_ShootingStar
  - Doji
  - Star


In [8]:
# Run backtest
backtest = Backtest(data=data, config=config)
results = backtest.run(strategies)

print("\nBacktest completed!")


Backtest completed!


In [9]:
# Compare results
comparison_df = results.compare()

print("\nCandlestick Strategy Comparison:")
print("=" * 100)
display_cols = ['total_return', 'cagr', 'sharpe_ratio', 'sortino_ratio', 'max_drawdown', 'total_trades', 'win_rate']
print(comparison_df[display_cols])


Candlestick Strategy Comparison:
                     total_return       cagr  sharpe_ratio  sortino_ratio  \
benchmark               50.191404  11.078280      0.509090       0.624592   
Star                     5.845905   1.478386     -0.077037      -0.094658   
Hammer_ShootingStar      2.923064   0.747007     -0.193096      -0.241656   
Doji                     2.753675   0.704151     -0.237788      -0.319713   
Engulfing                0.984678   0.253429     -0.324148      -0.436997   

                     max_drawdown  total_trades   win_rate  
benchmark               32.045757           1.0   0.000000  
Star                     9.826760          15.0  71.428571  
Hammer_ShootingStar     10.327332           8.0  50.000000  
Doji                     9.145510          14.0  71.428571  
Engulfing               12.646954          38.0  57.894737  


In [10]:
# Find best strategy
best_by_sharpe = results.best_strategy('sharpe_ratio')
best_by_return = results.best_strategy('total_return')

print(f"\nBest Strategy by Sharpe Ratio: {best_by_sharpe.name}")
print(f"  Sharpe: {best_by_sharpe.metrics['sharpe_ratio']:.2f}")
print(f"  Return: {best_by_sharpe.metrics['total_return']*100:.2f}%")

print(f"\nBest Strategy by Total Return: {best_by_return.name}")
print(f"  Return: {best_by_return.metrics['total_return']*100:.2f}%")
print(f"  Sharpe: {best_by_return.metrics['sharpe_ratio']:.2f}")


Best Strategy by Sharpe Ratio: Star
  Sharpe: -0.08
  Return: 584.59%

Best Strategy by Total Return: Star
  Return: 584.59%
  Sharpe: -0.08


In [11]:
# Visualize all strategies
fig = results.plot_comparison()
fig.update_layout(title="Candlestick Pattern Strategies Comparison")
fig.show()

## Detailed Analysis: Best Strategy

In [12]:
# Detailed metrics for best strategy
best_result = best_by_sharpe  # Already a StrategyResult object
metrics = best_result.metrics

print(f"\n{'='*70}")
print(f"Detailed Metrics: {best_result.name}")
print(f"{'='*70}")

print(f"\n📈 Returns:")
print(f"  Total Return:        {metrics['total_return']*100:>10.2f}%")
print(f"  CAGR:                {metrics['cagr']*100:>10.2f}%")

print(f"\n⚠️  Risk Metrics:")
print(f"  Sharpe Ratio:        {metrics['sharpe_ratio']:>10.2f}")
print(f"  Sortino Ratio:       {metrics['sortino_ratio']:>10.2f}")
print(f"  Calmar Ratio:        {metrics['calmar_ratio']:>10.2f}")
print(f"  Max Drawdown:        {metrics['max_drawdown']*100:>10.2f}%")
print(f"  Volatility:          {metrics['volatility']*100:>10.2f}%")

print(f"\n💰 Trade Statistics:")
print(f"  Total Trades:        {metrics['total_trades']:>10}")
print(f"  Win Rate:            {metrics['win_rate']*100:>10.2f}%")
print(f"  Profit Factor:       {metrics['profit_factor']:>10.2f}")
print(f"  Avg Win:             ${metrics['avg_win']:>10.2f}")
print(f"  Avg Loss:            ${metrics['avg_loss']:>10.2f}")

print(f"\n📊 Benchmark Comparison:")
print(f"  Alpha:               {metrics['alpha']*100:>10.2f}%")
print(f"  Beta:                {metrics['beta']:>10.2f}")
print(f"  Information Ratio:   {metrics['information_ratio']:>10.2f}")

print(f"\n💵 Final Values:")
print(f"  Initial Capital:     ${config.initial_capital:>10,.2f}")
print(f"  Final Value:         ${metrics['final_value']:>10,.2f}")
print(f"  Profit/Loss:         ${metrics['final_value']-config.initial_capital:>10,.2f}")
print(f"{'='*70}")


Detailed Metrics: Star

📈 Returns:
  Total Return:            584.59%
  CAGR:                    147.84%

⚠️  Risk Metrics:
  Sharpe Ratio:             -0.08
  Sortino Ratio:            -0.09
  Calmar Ratio:              0.15
  Max Drawdown:            982.68%
  Volatility:              501.27%

💰 Trade Statistics:
  Total Trades:                15
  Win Rate:               7142.86%
  Profit Factor:             1.62
  Avg Win:             $    205.97
  Avg Loss:            $   -318.22

📊 Benchmark Comparison:
  Alpha:                   -35.70%
  Beta:                      0.15
  Information Ratio:        -0.61

💵 Final Values:
  Initial Capital:     $ 10,000.00
  Final Value:         $ 10,584.59
  Profit/Loss:         $    584.59


## Summary

In this notebook, we explored candlestick pattern strategies:

1. ✅ **Engulfing Pattern**: Detects bullish and bearish engulfing candlesticks
2. ✅ **Hammer/Shooting Star**: Identifies reversal patterns based on shadow lengths
3. ✅ **Doji Pattern**: Trades indecision patterns with trend confirmation
4. ✅ **Morning/Evening Star**: Multi-candle reversal patterns
5. ✅ **Strategy Comparison**: Compared all candlestick strategies

### Key Insights:

- Candlestick patterns work best when combined with trend confirmation
- Different patterns have different success rates depending on market conditions
- Pattern recognition requires careful parameter tuning (e.g., body-to-shadow ratios)
- Combining multiple patterns can improve overall strategy performance

### Tips for Candlestick Trading:

1. **Context Matters**: Consider the overall trend when interpreting patterns
2. **Volume Confirmation**: High volume strengthens pattern signals
3. **Multiple Timeframes**: Verify patterns on different timeframes
4. **Risk Management**: Use stop-losses as patterns can fail
5. **Combine Indicators**: Use with technical indicators for better accuracy

### Next Steps:

- Combine candlestick patterns with technical indicators (RSI, MACD)
- Add volume analysis to pattern detection
- Test on different asset classes (forex, crypto)
- Create ensemble strategies combining multiple patterns